# Railway Train Debugger
Read the README.MD and FLOW.MD for more detailed explainations

## Configuration

In [1]:
import os
import shutil
import subprocess
import time
from CentralMock import *
centralMock = CentralMock()

## Single run simulations


In [ ]:
completedTaskDtos = []
completedTaskDtos.append(TaskDto())
completedTaskDtos[0].id = 1
completedTaskDtos[0].calculationStatus = 'COMPLETED'
completedTaskDtos[0].stationId = 2
completedTaskDtos[0].result = json.dumps({'calculation_result': 4})
completedTaskDtos[0].iteration = 0
completedTaskDtos.append(TaskDto())
completedTaskDtos[1].id = 2
completedTaskDtos[1].calculationStatus = 'COMPLETED'
completedTaskDtos[1].stationId = 3
completedTaskDtos[1].result =  json.dumps({'calculation_result': 6})
completedTaskDtos[1].iteration = 0

centralMock.clearTmpFolder()
centralMock.clearTrainFolder()

 # For the demo the input.txt is JSON format. But you are free to use any string format, as long as your applications knows what the format is.
print("run master final iteration")
inputStr = json.dumps({"iterations": 0})
outputStr, newTasks = centralMock.perfomMasterTask(inputStr, completedTaskDtos)
print(outputStr)
centralMock.archiveRun(True, 0, 0)
centralMock.clearTmpFolder()

print("run master some iteration")
inputStr = json.dumps({"iterations": 1})
outputStr, newTasks  = centralMock.perfomMasterTask(inputStr, completedTaskDtos)
print(newTasks)
centralMock.archiveRun(True, 99, 99)
centralMock.clearTmpFolder()

print("run station iteration")
inputStr = json.dumps({'number_to_process': 3})
outputStr = centralMock.perfomStationTask(inputStr)
print(outputStr)
centralMock.archiveRun(False, 0, 0)
centralMock.clearTmpFolder()


## perform computations for train
- Create a list of completed tasks for the stations you want to use 
- If you need more stations extend this list
- This module mocks the Central behaviour that triggers a train
- To demonstrate how the input works the number of iterations for this demo is set as an input variable for the prototype train

In [4]:
taskDtos = []
taskDtos.append(TaskDto())
taskDtos[0].id = 1
taskDtos[0].calculationStatus = 'COMPLETED'
taskDtos[0].stationId = 2
taskDtos[0].result = json.dumps({'calculation_result': 1})
taskDtos[0].iteration = 0
taskDtos.append(TaskDto())
taskDtos[1].id = 2
taskDtos[1].calculationStatus = 'COMPLETED'
taskDtos[1].stationId = 3
taskDtos[1].result =  json.dumps({'calculation_result': 2})
taskDtos[1].iteration = 0

centralMock.clearTmpFolder()
centralMock.clearTrainFolder()

ITERATIONS = 2
MASTER_STATION_ID = 1
TRAIN_DONE = False

 # For the demo the input.txt is JSON format. But you are free to use any string format, as long as your applications knows what the format is.
masterInputStr = json.dumps({"iterations": ITERATIONS})
# centralMock.clearTmpFolder()

iteration = 0
iteration_safety = 9 # break after 9 iterations, prevent infinite while
while not TRAIN_DONE:
    for taskDto in taskDtos:
        # loop over the tasks that should be done
        print('Starting Task:')
        print(json.dumps(taskDto.export()))
        if taskDto.calculationStatus != 'COMPLETED':
            taskDto.result = centralMock.perfomStationTask(taskDto.inputStr)
            taskDto.calculationStatus = 'COMPLETED'
            centralMock.archiveRun(False, taskDto.iteration, taskDto.stationId)
            #TODO add error handling, if error log is filled 

    print('starting master task for iteration: ' + str(taskDtos[0].iteration))
    masterOutputStr, newTaskDtos = centralMock.perfomMasterTask(masterInputStr, taskDtos)
    centralMock.archiveRun(True, taskDtos[0].iteration, MASTER_STATION_ID)

    if newTaskDtos.__len__() == 0:
        print('final master run detected')
        print(masterOutputStr)
        TRAIN_DONE = True
    else:
        print('new run detected, set the variables for the new run')
        taskDtos = []
        for item in newTaskDtos:
            taskDtos.append(TaskDto(item))
        masterInputStr = masterOutputStr
    
    iteration = iteration + 1
    if iteration > iteration_safety:
        print('error break out of while')
        TRAIN_DONE = True

        
print('simulation is done')

Starting Task:
{"result": "{\"calculation_result\": 1}", "input": "", "calculationStatus": "COMPLETED", "stationId": 2, "iteration": 0}
Starting Task:
{"result": "{\"calculation_result\": 2}", "input": "", "calculationStatus": "COMPLETED", "stationId": 3, "iteration": 0}
starting master task for iteration: 0
new run detected, set the variables for the new run
Starting Task:
{"result": "", "input": "{\"number_to_process\": 1}", "calculationStatus": "REQUESTED", "stationId": 2, "iteration": 1}
Starting Task:
{"result": "", "input": "{\"number_to_process\": 2}", "calculationStatus": "REQUESTED", "stationId": 3, "iteration": 1}
starting master task for iteration: 1
final master run detected
{"total_sum": 6.0}
simulation is done


## Final Results

In [ ]:
# import subprocess
# print(datetime.datetime.now())
# stream = subprocess.call('docker exec test_train sleep 5')

# # stream.wait()
# print(datetime.datetime.now())

# output = stream.stdout
# print(output)

In [ ]:
# test = centralMock.exportTaskDtoWithId(completedTasks[1])
# print(test)

In [ ]:
# COMPLETED_TASK_PATH = './tmp/completed-client-tasks.json'
# subprocess.call('docker cp ' + COMPLETED_TASK_PATH + ' test_train:/opt/completed-client-tasks.json', shell=True)

In [ ]:
if not os.path.lexists('./tmp/abc12/abc2/a'):
    os.makedirs('./tmp/abc12/abc2/a')
    

In [ ]:
print(completedTaskDtos[0].inputStr)

for dto in completedTaskDtos:
    dto.inputStr = 'test'

print(completedTaskDtos[0].inputStr)